#  Grocery Store Sales

FoodYum is a grocery store chain that is based in the United States.

Food Yum sells items such as produce, meat, dairy, baked goods, snacks, and other household food staples.

As food costs rise, FoodYum wants to make sure it keeps stocking products in all categories that cover a range of prices to ensure they have stock for a broad range of customers. 

## Data

The data is available in the table `products`.

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| product_type | Nominal. The product category type of the product, one of 5 values (Produce, Meat, Dairy, Bakery, Snacks). </br>Missing values should be replaced with “Unknown”. |
| brand | Nominal. The brand of the product. One of 7 possible values. </br>Missing values should be replaced with “Unknown”. |
| weight | Continuous. The weight of the product in grams. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median weight. |
| price | Continuous. The price the product is sold at, in US dollars. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median price. |
| average_units_sold | Discrete. The average number of units sold each month. This can be any positive integer value. </br>Missing values should be replaced with 0. |
| year_added | Nominal. The year the product was first added to FoodYum stock.</br>Missing values should be replaced with 2022. |
| stock_location | Nominal. The location that stock originates. This can be one of four warehouse locations, A, B, C or D </br>Missing values should be replaced with “Unknown”. |

# Finding missing value due to system bug

In 2022 there was a bug in the product system. For some products that were added in that year, the `year_added` value was not set in the data. As the year the product was added may have an impact on the price of the product, this is  information to have. 

PgSQL query to determine how many products have the `year_added` value missing. 

In [54]:
-- Write your query for task 1 in this cell
select count(*)  as  missing_year 
	from public.products 
	where 
	year_added is null;



,missing_year
0,170


# Data Cleaning and Imputing as per requirement

As we  know about the year added data missing, we need to make sure all of the data is clean before we start our analysis. The table below shows what the data should look like. 

Sql query to ensure that product data matches the description provided.   

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| product_type | Nominal. The product category type of the product, one of 5 values (Produce, Meat, Dairy, Bakery, Snacks). </br>Missing values should be replaced with “Unknown”. |
| brand | Nominal. The brand of the product. One of 7 possible values. </br>Missing values should be replaced with “Unknown”. |
| weight | Continuous. The weight of the product in grams. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median weight. |
| price | Continuous. The price the product is sold at, in US dollars. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median price. |
| average_units_sold | Discrete. The average number of units sold each month. This can be any positive integer value. </br>Missing values should be replaced with 0. |
| year_added | Nominal. The year the product was first added to FoodYum stock.</br>Missing values should be replaced with last year (2022). |
| stock_location | Nominal. The location that stock originates. This can be one of four warehouse locations, A, B, C or D </br>Missing values should be replaced with “Unknown”. |

## Check Column Data Types in PostgreSQL

To inspect the column names and their data types for a specific table within a schema, use the following PostgreSQL query: 

In [21]:
-- checking dtype of given schema and its table 
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'public' AND table_name = 'products';


,column_name,data_type
0,product_id,integer
1,product_type,text
2,brand,text
3,weight,text
4,price,real
5,average_units_sold,integer
6,year_added,integer
7,stock_location,text


For inspecting  the distinct and null value in table

In [40]:
-- finding distinct value in each columns
select distinct product_type from public.products;
select distinct brand from public.products;
select  price from public.products;
select average_units_sold from public.products ;
select year_added from public.products where year_added is null;
select distinct stock_location from public.products;

,stock_location
0,B
1,a
2,D
3,A
4,d
5,C
6,c
7,b


## calculating median value for imputing 

In [56]:
select 
	-- weight median
	percentile_cont(0.5) within group (order by trim(regexp_replace(weight, ' grams$',       ''))::numeric) as median_weight, 

	--price_median
	percentile_cont(0.5) within group(order by price) as median_price,

	--  last year
	max(year_added) as recent_year
	from 
	public.products

,median_weight,median_price,recent_year
0,500.92,8.95,2022


PostgreSQL code for imputing and Cleaning

In [ ]:

with pre_cal as
	(
	select 
	-- weight median
	percentile_cont(0.5) within group (order by trim(regexp_replace(weight, ' grams$',       ''))::numeric) as median_weight, 

	--price_median
	percentile_cont(0.5) within group(order by price) as median_price,

	--  last year
	max(year_added) as recent_year
	from 
	public.products
	
	)


	
select 
	 
	product_id,
	
	-- product_type
	 case 
	when product_type is null or product_type = '-' 
	    then 'Unknown' 
	else product_type end as product_type,
	
	-- brand
	case 
	when brand = '-' or brand is null then 'Unknown' else brand end  as brand,

	-- weight
	case 
	when trim(regexp_replace(weight, ' grams$', ''))::numeric is null 
	then 
	(select median_weight from pre_cal) 
	else trim(regexp_replace(weight, ' grams$', ''))::numeric end as weight,

	-- price
	case 
	when price is null then (select median_price from pre_cal)
	else price end as price,

	-- average_units_sold
	coalesce(average_units_sold ,0) as average_units_sold,

    -- year_added
	case
	when year_added is null  
	then 
	(select recent_year from pre_cal) else year_added end as year_added,

	-- stock_location
	case
	when stock_location = '-' or stock_location is null 
	then 'Unknown'
	else upper(stock_location) end as stock_location

from public.products


,product_id,product_type,brand,weight,price,average_units_sold,year_added,stock_location
0,1,Bakery,TopBrand,602.61,11.000000,15,2022,C
1,2,Produce,SilverLake,478.26,8.080000,22,2022,C
2,3,Produce,TastyTreat,532.38,6.160000,21,2018,B
3,4,Bakery,StandardYums,453.43,7.260000,21,2021,D
4,5,Produce,GoldTree,588.63,7.880000,21,2020,A
...,...,...,...,...,...,...,...,...
1695,1696,Meat,TastyTreat,503.99,14.080000,25,2017,A
1696,1697,Meat,GoldTree,526.89,16.129999,25,2016,D
1697,1698,Bakery,YumMie,583.85,7.050000,16,2021,A
1698,1699,Produce,TopBrand,441.64,8.100000,19,2019,A


# Analysing the min and max value acound each product type

To find out how the range varies for each product type,  to determine the minimum and maximum values for each product type.   

SQL query to return the `product_type`, `min_price` and `max_price` columns. 

In [42]:
-- Write your query for task 3 in this cell
select product_type , min(price) as min_price , max(price) as max_price
from public.products
group by product_type


,product_type,min_price,max_price
0,Snacks,5.20,10.72
1,Produce,3.46,8.78
2,Dairy,8.33,13.97
3,Bakery,6.26,11.88
4,Meat,11.48,16.98


# Analysis Around the certain specific product type for better understanding

require Detail at meat and dairy products where the average units sold was greater than ten. 

Sql query to return the `product_id`, `price` and `average_units_sold` of the rows of interest to the requirement. 

In [43]:
-- Write your query for task 4 in this cell
select product_id , price , average_units_sold
from public.products
where average_units_sold > 10 and (lower(product_type) = 'meat' or lower(product_type) = 'dairy')

,product_id,price,average_units_sold
0,6,16.20,24
1,8,15.77,28
2,9,11.57,30
3,10,13.94,27
4,11,9.26,26
...,...,...,...
693,1694,16.00,25
694,1695,12.88,20
695,1696,14.08,25
696,1697,16.13,25
